In [1]:
import numpy as np
import pandas as pd
import sys
from datetime import datetime
import os
from IPython.display import display

sys.path.insert(1, '/global/homes/b/bkieft/metatlas')
import notebooks.standards_library.standard_annotation as sta
pd.options.display.max_colwidth = 300

current_time = datetime.now().strftime("%Y%m%d%H%M%S")

# Enter Reference Standard information

In [2]:
ppm_tolerance = 5
include_polarities = ['POS', 'NEG']
include_chromatographies = ['C18', 'HILIC'] # 'C18' and/or 'HILIC'
include_adducts = ['[M+H]+', '[M+Na]+', '[M-H2O+H]+', '[M+K]+', '[M+NH4]+', '[M]+', '[M+2H]2+','[M-H]-', '[M+Cl]-', '[M]-', '[M-2H]2-']

path_to_standards_file = '/global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation'
standards_file = 'scheller_test.csv' # scheller_test.csv   metasci_flav_annotation_input.csv
standards_info_path = f'{path_to_standards_file}/{standards_file}'
input_compounds = pd.read_csv(standards_info_path)

current_ema_atlases = {"hilicz": {"pos": '/global/homes/b/bkieft/metatlas-data/HILIC/HILIC_EMA-standards_positive.tsv',
                                  "neg": '/global/homes/b/bkieft/metatlas-data/HILIC/HILIC_EMA-standards_negative.tsv'},
                        "c18": {"pos": '/global/homes/b/bkieft/metatlas-data/C18/C18_EMA-standards_positive.tsv',
                                  "neg": '/global/homes/b/bkieft/metatlas-data/C18/C18_EMA-standards_negative.tsv'}}
current_qc_atlases = {"hilicz": '/global/homes/b/bkieft/metatlas-data/HILIC/HILIC_QCv7_positive.tsv', # 0e0a13a57c434b258bf8ab33ab357961
                    "c18": '/global/homes/b/bkieft/metatlas-data/C18/C18_QCv7_positive.tsv'} # 20459fe5e4e14ab4a22168027b2bacda

current_msms_refs_path = '/global/cfs/cdirs/metatlas/projects/spectral_libraries/20240430_istdv7-addition_msms_refs.tab'

new_full_data = False
new_selected_data = False
new_filtered_data = True
new_rt_correction_data = True

# Get EIC and Spectra information from run table

In [3]:
standard_lcmsruns_table = sta.build_standard_lcmsrun_table(standards_info_path, include_polarities=include_polarities, include_chromatographies=include_chromatographies)
standard_lcmsruns_table_with_adducts = sta.build_adduct_annotated_table(standard_lcmsruns_table, include_adducts=include_adducts)
#standard_lcmsruns_table_with_adducts = standard_lcmsruns_table_with_adducts[standard_lcmsruns_table_with_adducts['compound_name'] == "sorgoleone"].head(1)

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 

[21:39:45] ERROR: 



In [4]:
if new_full_data is True:

    eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full = sta.extract_data(lcmsruns_table=standard_lcmsruns_table_with_adducts,
                                                                                                            ppm_tolerance=ppm_tolerance,
                                                                                                            method="find_peaks")
    
    runnum_to_structure_image_grid = sta.generate_gridded_molecular_images(standard_lcmsruns_table_with_adducts)

# Save and/or read full data

In [5]:
if new_full_data is True:
    sta.save_full_data(eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full, runnum_to_structure_image_grid, standards_info_path, current_time)
elif new_full_data is False:
    eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full, runnum_to_structure_image_grid = sta.load_full_data(standards_info_path)

Loading most recent pkl file: /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/scheller_test_20250422133530_ref_stds_data_full.pkl


# Create interactive plot and choose adducts

In [6]:
processed_data = sta.process_data_for_plotting(eics_full, top_spectra_full, group_names_full, rt_peaks_full, include_adducts)

processed_data = [
    entry for entry in processed_data
    if entry.get('compound_name') == "sorgoleone"
]

In [ ]:
selected_good_adducts = {}
ambiguous_adducts = {}

sta.create_interactive_plots(processed_data, runnum_to_structure_image_grid, selected_good_adducts, ambiguous_adducts)

In [23]:
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from rdkit.Chem import AllChem, Draw, MolFromSmiles
import base64

def generate_static_summary_image(processed_data, selected_good_adducts, current_index, export_dir):
    data = processed_data[current_index]
    eics = data['eics']
    rt_peaks = data['rt_peaks']
    top_spectra = data['top_spectra']
    adduct_color = data['adduct_color']
    group_id = data['group_id']
    unique_id = data['unique_id']

    rt_peaks['inchi'] = rt_peaks['smiles'].apply(lambda row: AllChem.MolToInchi(AllChem.MolFromSmiles(row)))
    rt_peaks['inchi_key'] = rt_peaks['inchi'].apply(sta.inchi_to_inchikey)
    rt_peaks['neutralized_inchi'] = rt_peaks['inchi'].apply(sta.neutralize_inchi)
    rt_peaks['neutralized_inchi_key'] = rt_peaks['neutralized_inchi'].apply(sta.inchi_to_inchikey)
    rt_peaks['permanent_charge'] = rt_peaks['neutralized_inchi'].apply(sta.charge_from_inchi)
    rt_peaks['formula'] = rt_peaks['neutralized_inchi'].apply(sta.formula_from_inchi)
    rt_peaks['monoisotopic_mass'] = rt_peaks['neutralized_inchi'].apply(sta.monoisotopic_mass_from_inchi)


    # Subset top_spectra by just the selected adducts and peak indices
    if unique_id in selected_good_adducts:
        selected_peaks = selected_good_adducts[unique_id]
        selected_adduct_peak_pairs = [
            (selected_peak.split("||")[0], selected_peak.split("||")[1])
            for selected_peak in selected_peaks
        ]
        top_spectra_subset = top_spectra[
            top_spectra.apply(
                lambda row: (row['adduct'], row['peak_index']) in selected_adduct_peak_pairs,
                axis=1
            )
        ]
    else:
        top_spectra_subset = pd.DataFrame()  # Empty DataFrame if no adducts are selected

    num_spectra = len(top_spectra_subset)
    num_columns = 3  # Increase to 3 columns to accommodate the table
    num_spectra_rows = math.ceil(num_spectra / (num_columns - 1))  # Adjust for the table column

    # Generate subplot titles
    eic_titles = ["Sample", "Sample (Log)"]
    spectra_titles = [
        f"{row['adduct']} @ {round(row['rt'], 2)} mins" for _, row in top_spectra_subset.iterrows()
    ]
    spectra_titles += [""] * (num_spectra_rows * (num_columns - 1) - len(spectra_titles))
    subplot_titles = eic_titles + ["Adduct Information"] + spectra_titles

    # Create specs for subplots
    specs = [[{"type": "scatter"}, {"type": "scatter"}, {"type": "table"}]] + \
            [[{"type": "scatter"}, {"type": "scatter"}, {"type": "image"}]] + \
            [[{"type": "scatter"}, {"type": "scatter"}, None]] * (num_spectra_rows - 1)

    # Ensure specs dimensions match the number of rows and columns
    while len(specs) < 2 + num_spectra_rows:
        specs.append([{"type": "scatter"}] * num_columns)

    fig = make_subplots(
        rows=2 + num_spectra_rows,
        cols=num_columns,
        shared_xaxes=False,
        vertical_spacing=0.3 / (2 + num_spectra_rows),
        horizontal_spacing=0.1,
        subplot_titles=subplot_titles,
        specs=specs
    )

    # Add EIC traces for "Sample" and "Sample (Log)"
    for lcmsrun_path, eic in eics.items():
        for _, eic_row in eic.iterrows():
            rt_sort = np.argsort(eic_row['rt'])
            adduct = sta.get_adduct(eic_row['label'])
            color = adduct_color.get(adduct, 'gray')

            # Add "Sample" trace
            fig.add_trace(
                go.Scatter(
                    x=eic_row['rt'][rt_sort],
                    y=eic_row['i'][rt_sort],
                    mode='lines',
                    name=f"{adduct} Sample",
                    line=dict(color=color)
                ),
                row=1,
                col=1
            )

            # Add "Sample (Log)" trace
            fig.add_trace(
                go.Scatter(
                    x=eic_row['rt'][rt_sort],
                    y=np.log10(eic_row['i'][rt_sort].astype(float)),
                    mode='lines',
                    name=f"{adduct} Sample (Log)",
                    line=dict(color=color)
                ),
                row=1,
                col=2
            )

            # Add rt_peak markers
            if not rt_peaks.empty:
                adduct_peaks = rt_peaks[rt_peaks['adduct'] == adduct]
                for _, peak_info in adduct_peaks.iterrows():
                    peak_rt = peak_info['rt_peak']
                    peak_intensity = peak_info['intensity']

                    # Add marker for "Sample"
                    fig.add_trace(
                        go.Scatter(
                            x=[peak_rt],
                            y=[peak_intensity],
                            mode='markers',
                            marker=dict(color=color, size=5),
                            name=f"{adduct} RT Peak",
                            showlegend=False
                        ),
                        row=1,
                        col=1
                    )

                    # Add marker for "Sample (Log)"
                    fig.add_trace(
                        go.Scatter(
                            x=[peak_rt],
                            y=[np.log10(peak_intensity)],
                            mode='markers',
                            marker=dict(color=color, size=5),
                            name=f"{adduct} RT Peak",
                            showlegend=False
                        ),
                        row=1,
                        col=2
                    )

                    if unique_id in selected_good_adducts:
                        selected_peaks = selected_good_adducts[unique_id]
                        for selected_peak in selected_peaks:
                            selected_adduct, peak_index = selected_peak.split("||")
                            if adduct == selected_adduct and peak_index == peak_info['peak_index']:
                                # Add star-shaped marker for "Sample"
                                fig.add_trace(
                                    go.Scatter(
                                        x=[peak_rt],
                                        y=[peak_intensity],
                                        mode='markers',
                                        marker=dict(color=color, size=14, symbol='star'),
                                        name=f"{adduct} Selected RT Peak",
                                        showlegend=False
                                    ),
                                    row=1,
                                    col=1
                                )

                                # Add star-shaped marker for "Sample (Log)"
                                fig.add_trace(
                                    go.Scatter(
                                        x=[peak_rt],
                                        y=[np.log10(peak_intensity)],
                                        mode='markers',
                                        marker=dict(color=color, size=14, symbol='star'),
                                        name=f"{adduct} Selected RT Peak (Log)",
                                        showlegend=False
                                    ),
                                    row=1,
                                    col=2
                                )

    # Add spectra plots for selected adducts and peak indices
    for i, (_, spectrum_row) in enumerate(top_spectra_subset.iterrows()):
        mz_values = spectrum_row['spectrum'][0]
        i_values = spectrum_row['spectrum'][1]
        adduct = spectrum_row['adduct']
        color = adduct_color.get(adduct, 'gray')

        spectrum_row_idx = 2 + (i // (num_columns - 1))
        spectrum_col = (i % (num_columns - 1)) + 1

        for mz, intensity in zip(mz_values, i_values):
            fig.add_trace(
                go.Scatter(
                    x=[mz, mz],
                    y=[0, intensity],
                    mode='lines',
                    line=dict(color=color),
                    showlegend=False
                ),
                row=spectrum_row_idx,
                col=spectrum_col
            )

        fig.add_trace(
            go.Scatter(
                x=mz_values,
                y=i_values,
                mode='markers',
                marker=dict(color=color, size=6),
                name=f"Spectrum {i+1}: {adduct}",
                showlegend=False
            ),
            row=spectrum_row_idx,
            col=spectrum_col
        )

    # Add table with adduct information
    if not rt_peaks.empty and unique_id in selected_good_adducts:
        selected_peaks = selected_good_adducts[unique_id]
        table_rows = []
        smiles = []
        for selected_peak in selected_peaks:
            selected_adduct, peak_index = selected_peak.split("||")
            rt_peaks_subset = rt_peaks[
                (rt_peaks['adduct'] == selected_adduct) &
                (rt_peaks['peak_index'] == peak_index)
            ]
            if not rt_peaks_subset.empty:
                table_rows.append(rt_peaks_subset)
                smiles.extend(rt_peaks_subset['smiles'].unique())

        if table_rows:
            combined_table = pd.concat(table_rows)
            table_data = {
                "Compound Name": combined_table['compound_name'],
                "Adduct": combined_table['adduct'],
                "Peak Index": combined_table['peak_index'],
                "Polarity": combined_table['polarity'],
                "MZ Observed": combined_table['mz_observed'],
                "MZ Theoretical": combined_table['mz_theoretical'],
                "Monoisotopic Mass": combined_table['monoisotopic_mass']
            }
            fig.add_trace(
                go.Table(
                    header=dict(
                        values=list(table_data.keys()),
                        fill_color='paleturquoise',
                        align='left'
                    ),
                    cells=dict(
                        values=[table_data[key] for key in table_data.keys()],
                        fill_color='lavender',
                        align='left'
                    )
                ),
                row=1,
                col=3
            )
        if smiles:
            if len(set(smiles)) == 1:
                smiles_string = smiles[0]
                smiles_df = pd.DataFrame({"smiles": [smiles_string]})
                molecular_image_dict = sta.generate_molecular_images(smiles_df)
                # Add the required prefix to the base64 string
                molecular_image = f"data:image/png;base64,{molecular_image_dict[smiles_string]}"
                fig.add_trace(
                    go.Image(
                        source=molecular_image,
                        x0=0,  # Starting x-coordinate
                        y0=0,  # Starting y-coordinate
                        dx=1,  # Scaling factor in the x-direction
                        dy=1   # Scaling factor in the y-direction
                    ),
                    row=2,
                    col=3
                )
                # Hide x and y axes for the image subplot
                fig.update_xaxes(visible=False, row=2, col=3)
                fig.update_yaxes(visible=False, row=2, col=3)
            else:
                print("Why are there two smiles for the same compound?")
                sys.exit(1)

    # Update layout
    fig.update_layout(
        title=f"{group_id} - Combined Plot",
        height=700 + 300 * num_spectra_rows,
        width=1200,
        plot_bgcolor="white",
        paper_bgcolor="white"
    )

    # Export the figure if export_dir is provided
    if export_dir:
        os.makedirs(export_dir, exist_ok=True)
        fig.write_image(
            f"{export_dir}/{group_id}_combined_plot.png",
            engine="kaleido",
            width=2000,
            height=1200,
            scale=2
        )

In [24]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
import base64
import math
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import plotly.io as pio

def create_interactive_plots(processed_data, runnum_to_structure_image_grid, \
                             selected_good_adducts, ambiguous_adducts, export_dir=None):

    # Widget Creation
    image_toggle = widgets.ToggleButton(
        value=False,  # Default to hidden
        description='Show Structures',
        tooltip='Toggle to show/hide the compound structure image',
        layout=widgets.Layout(width='150px', margin='30px 0 0 0')
    )
    yaxis_toggle = widgets.ToggleButton(
        value=False,  # Default to unique y-axis
        description='Shared Y-Axis',  # Description when toggled to shared y-axis
        tooltip='Toggle between unique and shared y-axes for non-log EIC plots',
        layout=widgets.Layout(width='150px', margin='30px 0 0 0')
    )
    next_button = widgets.Button(
        description="Next Group"
        )
    previous_button = widgets.Button(
        description="Previous Group"
        )
    progress_label = widgets.Label(
        value=f"1/{len(processed_data)} Groups Completed"
        )
    navigate_textbox = widgets.Text(
        placeholder='Index...',
        description='Go to:',
        layout=widgets.Layout(width='50px')
    )
    navigate_button = widgets.Button(
        description="Go",
        layout=widgets.Layout(width='50px')
    )
    compound_image_widget = widgets.Image(
        format='png',
        layout=widgets.Layout(
            width='400px',
            height='400px',
            margin='0 0 0 50px',
        )
    )
    output_container = widgets.Output()

    # Event Handlers
    def on_image_toggle_change(change):
        if image_toggle.value:
            image_toggle.description = 'Hide Structures'
            compound_image_widget.layout.display = 'block'
        else:
            image_toggle.description = 'Show Structures'
            compound_image_widget.layout.display = 'none'

    def update_progress_text():
        progress_label.value = f"{current_index + 1}/{len(processed_data)} Groups Completed"

    def on_toggle_change(change):
        yaxis_toggle.description = 'Shared Y-Axis' if not yaxis_toggle.value else 'Unique Y-Axis'
        update_plot(current_index)

    def navigate_to_group(b):
        nonlocal current_index
        try:
            target_index = int(navigate_textbox.value) - 1
            if 0 <= target_index < len(processed_data):
                current_index = target_index
                update_plot(current_index)
            else:
                output_container.clear_output(wait=True)
                with output_container:
                    print(f"Invalid index. Please enter a number between 1 and {len(processed_data)}.")
        except ValueError:
            output_container.clear_output(wait=True)
            with output_container:
                print("Invalid input. Please enter a valid integer.")

    def next_group(b):
        nonlocal current_index
        if current_index < len(processed_data) - 1:
            # Generate static summary image before moving to the next group
            generate_static_summary_image(processed_data, selected_good_adducts, current_index, export_dir=path_to_standards_file)
            
            current_index += 1
            update_plot(current_index)
            image_toggle.value = False
            image_toggle.description = 'Show Structures'
        else:
            output_container.clear_output(wait=True)
            with output_container:
                print("Analysis completed!")

    def previous_group(b):
        nonlocal current_index
        if current_index > 0:
            current_index -= 1
            print(f"Navigating to index {current_index}")  # Debug statement
            update_plot(current_index)
            # Reset the image toggle state
            image_toggle.value = False
            image_toggle.description = 'Show Structures'
        else:
            output_container.clear_output(wait=True)
            with output_container:
                print("Already at the first group.")

    # Attach Event Handlers
    image_toggle.observe(on_image_toggle_change, names='value')
    yaxis_toggle.observe(on_toggle_change, names='value')
    next_button.on_click(next_group)
    previous_button.on_click(previous_group)
    navigate_button.on_click(navigate_to_group)

    # Layout Definitions
    def create_layout(checkboxes):
        checkbox_layout = widgets.VBox(
            checkboxes,
            layout=widgets.Layout(
                border='1px solid black',
                padding='5px',
                margin='5px',
                width='325px',
                align_items='flex-start'
            )
        )
        # Layout of the "Go To:" widget
        go_to_label = widgets.Label(value="Go To:")
        go_to_layout = widgets.HBox(
            [go_to_label, navigate_textbox, navigate_button],
            layout=widgets.Layout(
                justify_content='flex-start',  # Align to the far left
                spacing='5px',
                margin='30px 0 0 0'  # Add space above the widget
            )
        )
        # Update the size of the search box
        navigate_textbox.description = ""
        navigate_textbox.layout = widgets.Layout(width='150px')  # Decrease the size of the search box

        compound_image_widget.layout.display = 'none'
        image_toggle.layout.margin = '0 0 0 50px'

        navigation_buttons_layout = widgets.HBox(
            [
                widgets.VBox([next_button, previous_button]),  # Stack Previous and Next buttons vertically
                image_toggle  # Place the Image Toggle button to the right
            ],
            layout=widgets.Layout(
                justify_content='flex-start',  # Align items to the left
                spacing='10px',  # Add spacing between elements
                margin='0 0 0 0'  # No margin for the navigation buttons
            )
        )
        button_layout = widgets.VBox(
            [navigation_buttons_layout, progress_label, go_to_layout, yaxis_toggle],
            layout=widgets.Layout(
                align_items='flex-start',
                spacing='5px'
            )
        )
        top_layout = widgets.HBox(
            [checkbox_layout, button_layout, compound_image_widget],
            layout=widgets.Layout(
                align_items='flex-start',
                justify_content='flex-start',
                spacing='10px'
            )
        )
        return top_layout

    # Plot Update Logic
    def update_plot(index, export_dir=export_dir):
        nonlocal current_index
        data = processed_data[index]

        eics = data['eics']
        top_spectra = data['top_spectra']
        rt_peaks = data['rt_peaks']
        adduct_color = data['adduct_color']
        group_id = data['group_id']
        unique_id = data['unique_id']
        group_run_number = data['group_run_number']

        # Extract adduct-peak combinations from rt_peaks and top_spectra
        adduct_peak_combinations = []
        if isinstance(rt_peaks, pd.DataFrame) and not rt_peaks.empty:
            # Create a mapping from adducts to peak indices and intensities
            adduct_to_peaks = {}
            for _, peak_row in rt_peaks.iterrows():
                adduct = peak_row['adduct'] if 'adduct' in peak_row else None
                if adduct:
                    if adduct not in adduct_to_peaks:
                        adduct_to_peaks[adduct] = []
                    adduct_to_peaks[adduct].append({
                        'peak_index': peak_row['peak_index'],
                        'intensity': peak_row['intensity']
                    })

        # Create unique identifiers for each adduct-peak combination
        for adduct, peaks in adduct_to_peaks.items():
            max_intensity = max(peak['intensity'] for peak in peaks)
            for peak in peaks:
                # Check if there is an MS2 spectrum for this adduct+peak_index
                if top_spectra.empty:
                    has_ms2 = False
                else:
                    has_ms2 = not top_spectra[
                        (top_spectra['adduct'] == adduct) & 
                        (top_spectra['peak_index'] == peak['peak_index'])
                    ].empty

                # Add a star to the description if MS2 spectrum exists
                description = f"{adduct} ({peak['peak_index']}){' *' if has_ms2 else ''}"
                adduct_peak_combinations.append({
                    'adduct': adduct,
                    'peak_index': peak['peak_index'],
                    'description': description,
                    'max_intensity': max_intensity
                })

        # Sort adduct_peak_combinations by max_intensity in descending order
        adduct_peak_combinations.sort(key=lambda x: x['max_intensity'], reverse=True)

        # Create the summary EIC plot data
        group_run_eics = [
            eic for pdata in processed_data if pdata['group_run_number'] == group_run_number
            for eic in pdata['eics'].values()
        ]
        summary_traces = []
        summary_xmin_list = []
        summary_xmax_list = []
        for eic in group_run_eics:
            # Loop through each row in the eic DataFrame
            for _, eic_row in eic.iterrows():
                # Filter data where intensity is above 1e5
                valid_indices = eic_row['i'] > 1e5
                filtered_rt = eic_row['rt'][valid_indices]
                filtered_i = eic_row['i'][valid_indices]

                if len(filtered_rt) > 0:  # Ensure there are valid points
                    # Sort retention times
                    rt_sort = np.argsort(filtered_rt)
                    adduct = sta.get_adduct(eic_row['label'])  # Extract adduct from the label
                    color = adduct_color.get(adduct, 'gray')  # Default to gray if adduct color is missing
                    label = eic_row['label']

                    # Update x_min and x_max based on filtered data
                    summary_xmin_list.append(filtered_rt.min())
                    summary_xmax_list.append(filtered_rt.max())

                    # Add a trace for the current adduct
                    summary_traces.append(
                        go.Scatter(
                            x=filtered_rt[rt_sort],
                            y=filtered_i[rt_sort],
                            mode='lines',
                            name=f"{label}",
                            line=dict(color=color),
                            showlegend=False
                        )
                    )
        x_min = min(summary_xmin_list) if summary_xmin_list else None
        x_max = max(summary_xmax_list) if summary_xmax_list else None

        # Create the figure with subplots
        num_spectra = len(top_spectra)
        if num_spectra == 0:
            num_spectra = 1  # Ensure at least one row for empty top_spectra
        num_columns = 4
        num_spectra_rows = math.ceil(num_spectra / num_columns)

        # Adjust subplot titles and specifications
        subplot_titles = [
            "Sample",
            "Blank",
            "EIC Summary",
            "Sample (Log)",
            "Blank (Log)",
            *(f"" if top_spectra.empty else f"{row['adduct']} @ {round(row['rt'], 2)} mins" for _, row in top_spectra.iterrows())
        ]

        specs = [
            [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter", "rowspan": 2, "colspan": 2}, None],
            [{"type": "scatter"}, {"type": "scatter"}, None, None],
            *[[{"type": "scatter"} for _ in range(4)] for _ in range(num_spectra_rows)]
        ]

        # Ensure there is at least one row for empty top_spectra
        if top_spectra.empty:
            subplot_titles.extend([""] * (num_spectra_rows * num_columns - len(subplot_titles) + 5))
            specs.extend([[{"type": "scatter"} for _ in range(4)] for _ in range(num_spectra_rows - 1)])

        fig = make_subplots(
            rows=2 + num_spectra_rows,
            cols=4,
            shared_xaxes=False,
            shared_yaxes=yaxis_toggle.value,
            vertical_spacing=0.3 / (2 + num_spectra_rows),
            horizontal_spacing=0.1,
            subplot_titles=subplot_titles,
            specs=specs
        )

        # Add fallback traces if top_spectra is empty
        if top_spectra.empty:
            fig.add_trace(
                go.Scatter(
                    x=[],
                    y=[],
                    mode='lines',
                    name="No Spectra Available",
                    line=dict(color='gray'),
                    showlegend=False
                ),
                row=3,
                col=1
            )

        # Add the summary traces to the spanning subplot
        fig.update_xaxes(range=[x_min, x_max], row=1, col=3)  # Set x-axis bounds for the summary graph
        for trace in summary_traces:
            fig.add_trace(trace, row=1, col=3)  # Add to row 1, col 3

        # Add EIC traces for each adduct/peak
        for idx, (lcmsrun_path, eic) in enumerate(eics.items()):
            for i, eic_row in eic.iterrows():
                rt_sort = np.argsort(eic_row['rt'])
                adduct = sta.get_adduct(eic_row['label'])
                color = adduct_color[adduct]
                
                # Determine row and column for the current trace
                row = 1 if idx < 2 else 2
                col = (idx % 2) + 1
                
                # Dynamic facet_name determination 
                if row == 1 and col == 1:
                    facet_name = "Sample"
                elif row == 1 and col == 2:
                    facet_name = "Blank"
                elif row == 2 and col == 1:
                    facet_name = "Sample (Log)"
                elif row == 2 and col == 2:
                    facet_name = "Blank (Log)"

                # Add line traces for raw intensity
                trace_index = len(fig.data)
                fig.add_trace(
                    go.Scatter(
                        x=eic_row['rt'][rt_sort],
                        y=eic_row['i'][rt_sort],
                        mode='lines',
                        name=f"{adduct} {facet_name}",  # Include facet_name in legend
                        line=dict(color=color),
                        showlegend=True
                    ),
                    row=row,
                    col=col
                )

                # Recalculate facet_name for log-transformed traces
                if row + 1 == 2 and col == 1:
                    facet_name = "Sample (Log)"
                elif row + 1 == 2 and col == 2:
                    facet_name = "Blank (Log)"

                # Add line traces for log-transformed intensity
                trace_index = len(fig.data)
                fig.add_trace(
                    go.Scatter(
                        x=eic_row['rt'][rt_sort],
                        y=np.log10(eic_row['i'][rt_sort].astype(float)),
                        mode='lines',
                        name=f"{adduct} {facet_name}",  # Include updated facet_name in legend
                        line=dict(color=color),
                        showlegend=True
                    ),
                    row=row + 1,  # Log traces go to the next row
                    col=col
                )

                # Add peak markers for each peak associated with this adduct
                if not rt_peaks.empty:
                    if facet_name == "Sample" or facet_name == "Sample (Log)":
                        adduct_peaks = rt_peaks[rt_peaks['adduct'] == adduct]
                        for _, peak_info in adduct_peaks.iterrows():
                            peak_rt = peak_info['rt_peak']
                            peak_index = peak_info['peak_index']
                            peak_intensity = peak_info['intensity']

                            # Add marker for raw intensity
                            fig.add_trace(
                                go.Scatter(
                                    x=[peak_rt],
                                    y=[peak_intensity],
                                    mode='markers',
                                    marker=dict(color=color, size=10),
                                    name=f"{adduct} RT {peak_index}",
                                    showlegend=False
                                ),
                                row=row,
                                col=col
                            )

                            # Add marker for log-transformed intensity
                            fig.add_trace(
                                go.Scatter(
                                    x=[peak_rt],
                                    y=[np.log10(peak_intensity)],
                                    mode='markers',
                                    marker=dict(color=color, size=10),
                                    name=f"{adduct} RT {peak_index}",
                                    showlegend=False
                                ),
                                row=row + 1,  # Log traces go to the next row
                                col=col
                            )

                # Add MS2 spectra markers
                if not top_spectra.empty:
                    if facet_name == "Sample" or facet_name == "Sample (Log)":
                        adduct_spectra = top_spectra[top_spectra['adduct'] == adduct]
                        # Remove adduct filtering to show all MS2 spectra
                        for _, spectrum_row in adduct_spectra.iterrows():
                            spectrum_adduct = spectrum_row['adduct']
                            spectrum_peak_index = spectrum_row['peak_index']
                            rounded_rt = round(spectrum_row['rt'], 2)
                            marker_color = adduct_color.get(spectrum_adduct, 'gray')

                            # Find closest point in the current EIC
                            sorted_rt = eic_row['rt'][rt_sort]
                            sorted_intensity = eic_row['i'][rt_sort]
                            
                            # Skip if no intensity data available
                            if len(sorted_rt) == 0 or len(sorted_intensity) == 0:
                                continue
                                
                            # Find the closest RT point in the EIC
                            closest_idx = np.argmin(np.abs(sorted_rt - spectrum_row['rt']))
                            
                            if closest_idx >= len(sorted_rt):
                                # If the index is out of bounds, skip this spectrum
                                print(f"Warning: RT {spectrum_row['rt']} is out of bounds for EIC RT range.")
                                continue
                                
                            raw_intensity = sorted_intensity[closest_idx]
                            log_intensity = np.log10(raw_intensity)
                            
                            # Display marker for all spectra on the raw intensity plot
                            fig.add_trace(
                                go.Scatter(
                                    x=[spectrum_row['rt']],
                                    y=[raw_intensity],
                                    mode='markers',
                                    marker=dict(color=marker_color, symbol='x', size=10),
                                    name=f"MS2: {spectrum_adduct} ({spectrum_peak_index}) @ {rounded_rt}",
                                    showlegend=False
                                ),
                                row=row,
                                col=col
                            )
                            
                            # Display marker for all spectra on the log-transformed plot
                            fig.add_trace(
                                go.Scatter(
                                    x=[spectrum_row['rt']],
                                    y=[log_intensity],
                                    mode='markers',
                                    marker=dict(color=marker_color, symbol='x', size=10),
                                    name=f"MS2: {spectrum_adduct} ({spectrum_peak_index}) @ {rounded_rt}",
                                    showlegend=False
                                ),
                                row=row + 1,
                                col=col
                            )

        # Add traces for Spectra plots
        if not top_spectra.empty:
            top_spectra_sorted = top_spectra.sort_values(['adduct', 'peak_index'])

            mz_list = [lst[0] for lst in top_spectra_sorted['spectrum'] if isinstance(lst, (list, np.ndarray)) and len(lst) > 0]
            mz_list_flattened = np.concatenate([np.ravel(arr) if isinstance(arr, np.ndarray) else np.array([arr]) for arr in mz_list])
            lowest_mz = np.min(mz_list_flattened)*0.9
            highest_mz = np.max(mz_list_flattened)*1.1

            for i, spectrum_row in enumerate(top_spectra_sorted.iterrows()):
                mz_values = spectrum_row[1]['spectrum'][0]
                i_values = spectrum_row[1]['spectrum'][1]
                adduct = spectrum_row[1]['adduct']
                color = adduct_color[adduct]
                precursor_mz = spectrum_row[1]['precursor_mz']
                peak_index = spectrum_row[1]['peak_index']
                spectrum_title = f"{adduct} ({peak_index}) @ {round(spectrum_row[1]['rt'], 2)} mins"

                # Determine the row and column for this spectrum
                spectrum_row_idx = 3 + (i // num_columns)  # Start after EIC rows
                spectrum_col = (i % num_columns) + 1

                # Update the x-axis range for the current subplot
                fig.update_xaxes(
                    range=[lowest_mz, highest_mz],  # Set x-axis limits
                    row=spectrum_row_idx,
                    col=spectrum_col
                )

                # Add vertical lines for each point
                for mz, intensity in zip(mz_values, i_values):
                    fig.add_trace(
                        go.Scatter(
                            x=[mz, mz],
                            y=[0, intensity],
                            mode='lines',
                            line=dict(color=color),
                            showlegend=False
                        ),
                        row=spectrum_row_idx,
                        col=spectrum_col
                    )

                # Add markers for each point
                fig.add_trace(
                    go.Scatter(
                        x=mz_values,
                        y=i_values,
                        mode='markers',
                        marker=dict(color=color, size=6),
                        name=f"Spectrum {i+1}: {adduct}",
                        showlegend=False
                    ),
                    row=spectrum_row_idx,
                    col=spectrum_col
                )

                # Add a black circle at precursor_mz (y=0)
                fig.add_trace(
                    go.Scatter(
                        x=[precursor_mz],
                        y=[0],
                        mode='markers',
                        marker=dict(color='black', symbol='circle', size=20),
                        name=f"Precursor MZ: {precursor_mz}",
                        showlegend=False
                    ),
                    row=spectrum_row_idx,
                    col=spectrum_col
                )
                
                fig.layout.annotations[5 + i].text = spectrum_title

        # Update layout
        fig_title = (group_id.replace('_', '  |  '))
        fig.update_layout(
            hoverlabel=dict(
                font_size=11,  # Increase font size for better readability
                namelength=-1  # Show the full name without truncation
            ),
            title=dict(text=fig_title, font=dict(size=14), x=0.5, xanchor="center"),
            height=700 + 300 * num_spectra_rows,
            width=1500,
            plot_bgcolor="white",
            paper_bgcolor="white",
            legend=dict(
                orientation="h",
                xanchor="center",
                yanchor="top",
                x=0.5,
                y=-0.2
            )
        )

        # Add black borders and keep gridlines
        fig.update_xaxes(
            showline=True,  # Show axis line
            linewidth=1,  # Set line width
            linecolor="black",  # Set line color to black
            showgrid=True,  # Keep gridlines
            gridcolor="lightgray"  # Set gridline color
        )
        fig.update_yaxes(
            showline=True,  # Show axis line
            linewidth=1,  # Set line width
            linecolor="black",  # Set line color to black
            showgrid=True,  # Keep gridlines
            gridcolor="lightgray"  # Set gridline color
        )


        # Update the compound image based on group_run_number
        if group_run_number in runnum_to_structure_image_grid:
            compound_image_widget.value = base64.b64decode(runnum_to_structure_image_grid[group_run_number])
        else:
            compound_image_widget.value = b''  # Clear the image if not found

        # Create checkboxes for selecting good adducts with peak indices
        checkboxes = [
            widgets.Checkbox(
                value=False,
                description=combo['description'],
                disabled=False
            )
            for combo in adduct_peak_combinations
        ]
        ambiguous_checkbox = widgets.Checkbox(
            value=False,
            description="Ambiguous",
            disabled=False
        )
        checkboxes.append(ambiguous_checkbox)
        checkbox_dict = {checkbox.description: checkbox for checkbox in checkboxes}

        def on_checkbox_change(change):
            if ambiguous_checkbox.value:
                ambiguous_adducts[unique_id] = unique_id
                selected_good_adducts.pop(unique_id, None)
            else:
                selected_good_adducts[unique_id] = [
                    combo['adduct'] + "||" + str(combo['peak_index'])
                    for combo in adduct_peak_combinations 
                    if checkbox_dict[combo['description']].value
                ]
                ambiguous_adducts.pop(unique_id, None)

        for checkbox in checkboxes:
            checkbox.observe(on_checkbox_change, names='value')

        # Set previously selected values
        if unique_id in selected_good_adducts:
            for selected_combo in selected_good_adducts[unique_id]:
                adduct, peak_index = selected_combo.split("||")
                for combo in adduct_peak_combinations:
                    if combo['adduct'] == adduct and str(combo['peak_index']) == peak_index:
                        combo_description = combo['description']
                        if combo_description in checkbox_dict:
                            checkbox_dict[combo_description].value = True
        elif unique_id in ambiguous_adducts:
            ambiguous_checkbox.value = True

        # Create the layout with checkboxes
        top_layout = create_layout(checkboxes)
        plot_and_checkboxes = widgets.VBox(
            [top_layout, widgets.Output()],
            layout=widgets.Layout(align_items='flex-start')
        )

        clear_output(wait=True)
        update_progress_text()
        with plot_and_checkboxes.children[1]:
            display(fig)
        display(plot_and_checkboxes)
        display(output_container)

    # Initialize
    current_index = 0
    update_plot(current_index)


In [25]:
selected_good_adducts = {}
ambiguous_adducts = {}
processed_data = sta.process_data_for_plotting(eics_full, top_spectra_full, group_names_full, rt_peaks_full, include_adducts)

create_interactive_plots(processed_data, runnum_to_structure_image_grid, selected_good_adducts, ambiguous_adducts, export_dir=path_to_standards_file)

Output()

In [34]:
smiles_df = pd.DataFrame({"smiles": ['COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\\\\C/C=C\\\\CC=C)O']})
smiles_df

,smiles
0,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\\C/C=C\\CC=C)O


# Save and/or read selected data

In [8]:
if new_selected_data is True:
    sta.save_selected_data(selected_good_adducts, ambiguous_adducts, standards_info_path, current_time)
elif new_selected_data is False:
    selected_good_adducts, ambiguous_adducts = sta.load_selected_data(standards_info_path)

Saving data to: /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/scheller_test_20250423144108_ref_stds_data_selected.pkl


# Preview compounds+adducts selections

In [9]:
selected_compounds_table = sta.extract_selected_compounds(selected_good_adducts)
ambiguous_compounds_table = sta.extract_ambiguous_compounds(ambiguous_adducts)

print("Selected compounds:\n")
display(selected_compounds_table)
print("\nAmbiguous compounds (to return to for scrutiny):\n")
display(ambiguous_compounds_table)

Selected compounds:



,compound_name,standard_lcmsrun,selected_adducts,selected_peak_indices
0,sorgoleone,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_POS_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run375.h5,"[[M+H]+, [M+NH4]+]","[peak1, peak1]"
1,sorgoleone,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_POS_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE205060norm-200uM-S1_Run376.h5,"[[M+H]+, [M+NH4]+]","[peak1, peak1]"
2,sorgoleone,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_NEG_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run377.h5,[[M-H]-],[peak1]
3,sorgoleone,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_NEG_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE205060norm-200uM-S1_Run378.h5,[[M-H]-],[peak1]
4,sorgoleone,/global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782_POS_MS2_RefStd-1_Sorgoleone_1_Rg80to1200-CE102040norm-200uM-S1_Run382.h5,"[[M+Na]+, [M+H]+]","[peak1, peak1]"
5,sorgoleone,/global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782_POS_MS2_RefStd-1_Sorgoleone_1_Rg80to1200-CE205060norm-200uM-S1_Run383.h5,"[[M+Na]+, [M+H]+]","[peak1, peak1]"
6,sorgoleone,/global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782_NEG_MS2_RefStd-1_Sorgoleone_1_Rg80to1200-CE102040norm-200uM-S1_Run384.h5,[[M-H]-],[peak1]
7,sorgoleone,/global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782_NEG_MS2_RefStd-1_Sorgoleone_1_Rg80to1200-CE205060norm-200uM-S1_Run385.h5,[[M-H]-],[peak1]



Ambiguous compounds (to return to for scrutiny):



""


# Filter RT Peak, EICs, and Top Spectra by selected compounds+adducts

In [10]:
eics_filtered, rt_peaks_filtered, top_spectra_filtered = sta.filter_by_selected(eics_full, rt_peaks_full, top_spectra_full, selected_compounds_table)

print(f"\nTotal unique compounds retained: {eics_filtered['compound_name'].nunique()}")
print(f"Total unique compound+adduct entries retained: {eics_filtered['label'].nunique()}\n")
print(f"Total EICs selected: {eics_filtered.shape[0]}")
print(f"Total RT peaks selected: {rt_peaks_filtered.shape[0]}")
print(f"Total MS2 spectra selected: {top_spectra_filtered.shape[0]}")

[14:43:23] non-ring atom 0 marked aromatic
[14:43:23] non-ring atom 0 marked aromatic



Total unique compounds retained: 1
Total unique compound+adduct entries retained: 10

Total EICs selected: 31
Total RT peaks selected: 12
Total MS2 spectra selected: 12


# Save and/or read filtered data

In [11]:
if new_filtered_data is True:
    sta.save_filtered_data(eics_filtered, top_spectra_filtered, rt_peaks_filtered, standards_info_path, current_time)
elif new_filtered_data is False:
    eics_filtered, top_spectra_filtered, rt_peaks_filtered = sta.load_filtered_data(standards_info_path)

Saving data to: /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/scheller_test_20250423144108_ref_stds_data_filtered.pkl


# Save summary plots of selected compounds+adducts

In [12]:
# TO DO:
# For each compound (group_id in the GUI), export the full EIC (log/normal) and indicate which peak+msms was chosen - same format as mirror plots (ish)
# Inlude an image of the compound and a table of other information like mz, formula, mass
# And mark which peaks were chosen for each adduct

# Choose only the top adduct per compound to add to DB, Atlases, and MSMS Refs

In [13]:
rt_peaks_filtered_top_adduct = sta.filter_top_compounds(rt_peaks_filtered)


Filtered 12 compound peaks to 4 peaks by best adduct. Here are the compounds+adducts retained:



,label,adduct,polarity,chromatography,inchi_key,monoisotopic_mass
3,sorgoleone,[M+H]+,POS,HILICZ,FGWRUVXUQWGLOX-AFJQJTPPSA-N,358.214409
1,sorgoleone,[M+Na]+,POS,C18,FGWRUVXUQWGLOX-AFJQJTPPSA-N,358.214409
0,sorgoleone,[M-H]-,NEG,C18,FGWRUVXUQWGLOX-AFJQJTPPSA-N,358.214409
2,sorgoleone,[M-H]-,NEG,HILICZ,FGWRUVXUQWGLOX-AFJQJTPPSA-N,358.214409


# Check if selected compounds are in metatlas DB

In [14]:
in_db, notin_db = sta.search_for_matches_in_metatlas_db(rt_peaks_filtered_top_adduct, check_by_flat=True)

Searching for matches in MSMS refs:   0%|          | 0/4 [00:00<?, ?it/s]


These compounds are not yet in the metatlas database:



,label,inchi,inchi_key,neutralized_inchi,neutralized_inchi_key,permanent_charge,formula,monoisotopic_mass
0,sorgoleone,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",FGWRUVXUQWGLOX-AFJQJTPPSA-N,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",FGWRUVXUQWGLOX-AFJQJTPPSA-N,0,C22H30O4,358.214409


# Store selected compounds+adducts in metatlas db

In [15]:
#sta.store_in_metatlas_db(notin_db)

# Check if selected compounds/adducts are in the atlases

In [16]:
ema_atlases_data = sta.get_ema_atlas_data(current_ema_atlases)

In [17]:
rt_peaks_filtered_top_adduct_formatted = sta.convert_rt_peaks_to_atlas_format(rt_peaks_filtered_top_adduct)

In [18]:
matches_to_atlases, nonmatches_to_atlases = sta.search_for_matches_in_atlases(rt_peaks_filtered_top_adduct_formatted, ema_atlases_data)

Searching for matches in existing atlases:   0%|          | 0/4 [00:00<?, ?it/s]


These compounds+adducts are not yet in any atlases:



,standard_lcmsrun,chromatography,compound_name,adduct,polarity,rt_peak,mz,smiles,peak_index,inchi,...,permanent_charge,formula,mono_isotopic_molecular_weight,collision_energy,label,rt_min,rt_max,mz_tolerance,mz_tolerance_units,in_metatlas
0,/global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782_POS_MS2_RefStd-1_Sorgoleone_1_Rg80to1200-CE205060norm-200uM-S1_Run383.h5,C18,sorgoleone,[M+Na]+,positive,7.544277,381.203627,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",...,0,C22H30O4,358.214409,205060norm,sorgoleone,7.044277,8.044277,5,ppm,True
1,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_NEG_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run377.h5,HILICZ,sorgoleone,[M-H]-,negative,0.804263,357.207133,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",...,0,C22H30O4,358.214409,102040norm,sorgoleone,0.304263,1.304263,5,ppm,True
2,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_POS_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run375.h5,HILICZ,sorgoleone,[M+H]+,positive,0.801184,359.221685,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",...,0,C22H30O4,358.214409,102040norm,sorgoleone,0.301184,1.301184,5,ppm,True


In [19]:
if new_rt_correction_data is True:

    baseline_to_experimental_qc = sta.get_qc_experimental_atlas(nonmatches_to_atlases, current_qc_atlases, include_istds=True)

Getting raw QC files for /global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782...

Retrieving baseline C18 QC atlas...



KeyError: 'C18'

In [ ]:
if new_rt_correction_data is True:
    sta.save_rt_correction_data(baseline_to_experimental_qc, standards_info_path, current_time)
elif new_rt_correction_data is False:
    baseline_to_experimental_qc = sta.load_rt_correction_data(standards_info_path)

In [ ]:
baseline_correction_inputs = sta.create_baseline_correction_input(nonmatches_to_atlases, baseline_to_experimental_qc)
baseline_correction_outputs = sta.rt_correction_from_baseline(baseline_correction_inputs)

In [ ]:
nonmatches_to_atlases_rt_corrected = sta.substitute_corrected_rt_values(nonmatches_to_atlases, baseline_correction_outputs)
print(f"Inserting {nonmatches_to_atlases_rt_corrected.shape[0]} RT-corrected compounds into the database")

# Store selected compound/adduct in metatlas atlases

In [ ]:
sta.update_and_save_atlases(ema_atlases_data, nonmatches_to_atlases_rt_corrected, current_time, path_to_standards_file, save_atlas=True)

# Check if selected compounds/adducts are in MSMS refs

In [ ]:
msms_refs = sta.get_msms_refs(msms_refs_path=current_msms_refs_path)
msms_refs.shape

In [ ]:
in_msms_refs, notin_msms_refs = sta.search_for_matches_in_msms_refs(rt_peaks_filtered_top_adduct_formatted, msms_refs, check_by_flat=True)

In [ ]:
rt_peaks_filtered_with_top_spectra = sta.merge_selected_peaks_with_top_spectra(notin_msms_refs, top_spectra_filtered)

In [ ]:
rt_peaks_filtered_with_top_spectra_formatted = sta.format_for_msms_refs(rt_peaks_filtered_with_top_spectra, msms_refs)

# Store selected compound/adduct in MSMS refs

In [ ]:
sta.update_and_save_msms_refs(msms_refs, rt_peaks_filtered_with_top_spectra_formatted, path_to_standards_file, current_time, save_refs=True)

# Check to verify that compounds were added to database, atlases, and refs correctly

In [ ]:
# TO DO: Add a checker